In [97]:
import pandas as pd
import random

LEVEL_CONFIG = {1:2, 2:3, 3:4, 4:5, 5:5}

def decidir_accion(nivel, accuracy, tiempo_por_acierto, racha):
    # BAJAR si desempeño es claramente malo
    if accuracy < 0.6 and racha <= -2:
        return "BAJAR"

    # SUBIR si desempeño es claramente bueno
    if accuracy >= 0.95 and tiempo_por_acierto < (2.2 + nivel * 0.4) and racha >= 2:
        return "SUBIR"

    return "MANTENER"


data = []
racha = 0

for _ in range(6000):

    nivel = random.randint(1, 5)
    largo = LEVEL_CONFIG[nivel]

    # Errores más realistas
    max_errores = min(largo, random.choice([1, 2]) if nivel >= 3 else 1)
    errores = random.randint(0, max_errores)

    aciertos = max(0, largo - errores)
    accuracy = aciertos / largo if largo > 0 else 0

    # Tiempo realista
    if errores == 0:
        tiempo_total = random.uniform(largo * 1.1, largo * 2.0)
    else:
        tiempo_total = random.uniform(largo * 1.8, largo * 3.5)

    tiempo_por_acierto = tiempo_total / max(1, aciertos)

    # Racha
    if errores == 0:
        racha += 1
    else:
        racha -= 1
    racha = max(-5, min(5, racha))

    accion = decidir_accion(nivel, accuracy, tiempo_por_acierto, racha)

    data.append([
        nivel, largo, aciertos, errores,
        round(accuracy, 2),
        round(tiempo_por_acierto, 2),
        racha,
        accion
    ])

df = pd.DataFrame(data, columns=[
    "Nivel", "Largo_Patron", "Aciertos", "Errores",
    "Accuracy", "Tiempo_por_Acierto", "Racha", "Accion"
])

df.to_csv("dataset_simon.csv", index=False)

df


,Nivel,Largo_Patron,Aciertos,Errores,Accuracy,Tiempo_por_Acierto,Racha,Accion
0,5,5,4,1,0.8,3.32,-1,MANTENER
1,4,5,4,1,0.8,3.92,-2,MANTENER
2,1,2,2,0,1.0,1.28,-1,MANTENER
3,3,4,2,2,0.5,6.43,-2,BAJAR
4,1,2,2,0,1.0,1.72,-1,MANTENER
...,...,...,...,...,...,...,...,...
5995,5,5,5,0,1.0,1.48,-4,MANTENER
5996,2,3,3,0,1.0,1.31,-3,MANTENER
5997,1,2,1,1,0.5,5.35,-4,BAJAR
5998,3,4,4,0,1.0,1.41,-3,MANTENER


In [98]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import joblib

df = pd.read_csv("dataset_simon.csv")

X = df.drop(columns=["Accion"])
y = df["Accion"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

modelo = RandomForestClassifier(n_estimators=200, max_depth=10)
modelo.fit(X_train, y_train)

print("Accuracy:", modelo.score(X_test, y_test))

joblib.dump(modelo, "modelo_rf_new.pkl")
print("Modelo guardado.")


Accuracy: 1.0
Modelo guardado.


In [110]:
#['Nivel', 'Largo_Patron', 'Aciertos', 'Errores', 'Tiempo_Reaccion', 'Racha']
import numpy as np

test = pd.DataFrame([{
    'Nivel': 3,
    'Largo_Patron' : 4,
    'Aciertos': 3,
    'Errores': 1,
    'Accuracy' : 1,
    'Tiempo_por_Acierto': 10,
    'Racha': 4
}])
res = modelo.predict(test)
print(test,res)

   Nivel  Largo_Patron  Aciertos  Errores  Accuracy  Tiempo_por_Acierto  Racha
0      3             4         3        1         1                  10      4 ['MANTENER']
